### Initial Setup
Make sure you have the following packages installed:
- pandas
- urllib3
- requests
- datetime

`pip install pandas urllib3 requests datetime`

In [1]:
# pip install pandas urllib3 requests datetime

In [2]:
import pandas as pd
import os
from datetime import date, timedelta

# import our functions
import api_defillama as dfl 
import api_growthe as gtp 
import api_l2beat as l2b

In [3]:
# set filter variables - to help cut down on rows for this demo
trailing_days = 365*5
get_start_of_month = True # True if we want to start on the first day of the month

In [4]:
# compute date range
current_date = date.today()
start_date = current_date - timedelta(days = trailing_days)
if get_start_of_month:
    start_date = start_date.replace(day = 1)
#Format
current_date = pd.to_datetime(current_date)
start_date = pd.to_datetime(start_date)

print(f"Today's Date: {current_date}, Filter Start Date: {start_date}")

Today's Date: 2024-11-07 00:00:00, Filter Start Date: 2019-11-01 00:00:00


### Run Growthepie
[Growthepie](https://www.growthepie.xyz/) provides detailed onchain data (i.e. transactions, revenue, costs, profit) for a set of L2s.

See [API Docs](https://docs.growthepie.xyz/api)

In [5]:
print('getting growthepie metadata')
gtp_md = gtp.get_growthepie_metadata()

print('getting growthepie fundamentals data')
gtp_fun = gtp.get_growthepie_fundamentals()
#filter dates
gtp_fun = gtp_fun[gtp_fun['date']>=start_date]

print('yay')

getting growthepie metadata
getting growthepie fundamentals data
https://api.growthepie.xyz/v1/fundamentals_full.json
yay


In [6]:
num_chains = len(gtp_md[gtp_md['chain_name']!= '-'])
print(f"Growthepie: Total Amount of Chains: {num_chains}")

Growthepie: Total Amount of Chains: 24


In [7]:
gtp_md.head(5)

,name,url_key,chain_type,caip2,evm_chain_id,deployment,chain_name,description,da_layer,symbol,...,l2beat_id,raas,stack,website,twitter,block_explorer,block_explorers,rhino_listed,rhino_naming,origin_key
0,Ethereum,ethereum,L1,eip155-1,1.0,PROD,Ethereum,"Ethereum, proposed by Vitalik Buterin in 2013 ...",None,ETH,...,ethereum,Self-hosted,"{'label': 'Custom', 'url': None}",https://ethereum.org/,https://twitter.com/ethereum,https://etherscan.io/,"{'Etherscan': 'https://etherscan.io/', 'Blocks...",True,ETHEREUM,ethereum
1,All L2s,all-l2s,-,None,NaN,PROD,-,None,None,None,...,None,None,None,None,None,None,None,False,None,all_l2s
2,Arbitrum One,arbitrum,L2,eip155-42161,42161.0,PROD,Arbitrum,Arbitrum One is developed by Offchain Labs. It...,Ethereum (blobs),ARB,...,arbitrum,Self-hosted,"{'label': 'Arbitrum Nitro', 'url': 'https://ar...",https://arbitrum.io/,https://twitter.com/arbitrum,https://arbiscan.io/,"{'Arbiscan': 'https://arbiscan.io/', 'Blocksco...",True,ARBITRUM,arbitrum
3,Base,base,L2,eip155-8453,8453.0,PROD,Base,"Base is an Ethereum Layer 2 offering a secure,...",Ethereum (blobs),None,...,base,Self-hosted,"{'label': 'OP Stack, Superchain', 'url': 'http...",https://base.org/,https://twitter.com/base,https://basescan.org/,"{'BaseScan': 'https://basescan.org/', 'Blocksc...",True,BASE,base
4,Blast,blast,L2,eip155-81457,81457.0,PROD,Blast,Blast is an EVM-compatible Optimistic Rollup w...,Ethereum (blobs),BLAST,...,blast,Self-hosted,"{'label': 'OP Stack', 'url': 'https://docs.opt...",https://blast.io/en,https://twitter.com/Blast_L2,https://blastscan.io/,{'Blast Explorer': 'https://blastscan.io/'},True,BLAST,blast


In [8]:
gtp_fun[gtp_fun['date']>='2024-10-01'].sample(5)

metric_key,index,origin_key,date,costs_blobs_eth,costs_blobs_usd,costs_l1_eth,costs_l1_usd,costs_total_eth,costs_total_usd,daa,...,profit_usd,rent_paid_eth,rent_paid_usd,stables_mcap,stables_mcap_eth,tvl,tvl_eth,txcosts_median_eth,txcosts_median_usd,txcount
11026,14137,redstone,2024-10-05,NaN,NaN,0.006188,14.955932,0.006188,14.955932,153.0,...,NaN,0.006188,14.955932,2.276671e+04,9.419728e+00,1.534823e+06,6.350330e+02,9.213138e-07,0.002227,2394.0
1882,3101,blast,2024-10-16,1.069548e-10,2.783044e-07,0.075370,196.117414,0.075370,196.117414,24993.0,...,3610.549368,0.075370,196.117414,2.306088e+08,8.862494e+04,1.458604e+09,5.605541e+05,9.192593e-07,0.002392,690160.0
10825,13881,polygon_zkevm,2024-10-22,NaN,NaN,2.313101,6166.106639,2.313101,6166.106639,2751.0,...,-5117.285515,2.313101,6166.106639,2.377777e+07,8.919789e+03,6.654166e+07,2.496187e+04,8.757000e-06,0.023344,20347.0
1539,2069,base,2024-10-25,4.974445e-09,1.261636e-05,0.308635,782.770181,0.308635,782.770193,1393405.0,...,165556.094338,0.308635,782.770193,3.463546e+09,1.365626e+06,7.691125e+09,3.032498e+06,9.055114e-07,0.002297,6428603.0
1544,2074,base,2024-10-30,1.412716e+00,3.727645e+03,0.571183,1507.145200,1.983899,5234.790628,1248930.0,...,153679.067603,1.983899,5234.790628,3.548663e+09,1.344885e+06,8.102952e+09,3.070885e+06,1.155294e-06,0.003048,6977287.0


### Run L2Beat
[L2Beat](https://l2beat.com/) provides high-level metrics (transactions, assets onchain) and security evaluations for a wide set of L2s and L3s.

In [9]:
print('getting l2beat summary data')
l2b_summary = l2b.get_l2beat_summary()
print('getting l2beat activity data')
l2b_activity = l2b.get_all_projects_data(l2b_summary, 'activity',query_range='max')

print('getting l2beat assets onchain data')
l2b_assets = l2b.get_all_projects_data(l2b_summary, 'tvl',query_range='max')

#filter dates
l2b_activity = l2b_activity[l2b_activity['timestamp']>=start_date]
l2b_assets = l2b_assets[l2b_assets['timestamp']>=start_date]
print('yay')

getting l2beat summary data
https://l2beat.com/api/scaling/summary
getting l2beat activity data
activity API, Chains to run: 115
25 / 115 completed
50 / 115 completed
75 / 115 completed
100 / 115 completed
All projects completed
getting l2beat assets onchain data
tvl API, Chains to run: 115
25 / 115 completed
50 / 115 completed
75 / 115 completed
100 / 115 completed
All projects completed
yay


In [10]:
num_chains = len(l2b_summary[(l2b_summary['isArchived']==False) &(l2b_summary['isUpcoming']==False)])
print(f"L2Beat: Total Amount of Chains: {num_chains}")
l2b_summary.sample(5)

L2Beat: Total Amount of Chains: 115


,id,name,slug,type,category,provider,purposes,isArchived,isUpcoming,isUnderReview,badges,tvl,stage,risks,shortName,hostChain
100,clique,Clique,clique,layer3,Optimium,OP Stack,"[AI, Gaming]",False,False,True,"[{'category': 'L3ParentChain', 'name': 'Base'}...","{'breakdown': {'total': 1863.98, 'ether': 1863...",NotApplicable,"[{'name': 'Sequencer Failure', 'value': 'Under...",NaN,Base
93,lambda,Lambda Chain,lambda,layer2,Optimistic Rollup,OP Stack,"[Universal, Storage]",False,False,False,"[{'category': 'Stack', 'name': 'OPStack'}, {'c...","{'breakdown': {'total': 14170.25, 'ether': 141...",Stage 0,"[{'name': 'Sequencer Failure', 'value': 'Self ...",NaN,NaN
47,silicon,Silicon,silicon,layer2,Validium,Polygon,[Universal],False,False,False,"[{'category': 'DA', 'name': 'DAC'}, {'category...","{'breakdown': {'total': 13168448.72, 'ether': ...",NotApplicable,"[{'name': 'Sequencer Failure', 'value': 'No me...",NaN,NaN
82,parallel,Parallel,parallel,layer2,Optimistic Rollup,Arbitrum,[Universal],False,False,False,"[{'category': 'Stack', 'name': 'Orbit'}, {'cat...","{'breakdown': {'total': 75988.46, 'ether': 759...",Stage 0,"[{'name': 'Sequencer Failure', 'value': 'Self ...",NaN,NaN
51,winr,WINR,winr,layer3,Optimium,Arbitrum,"[Universal, Gaming]",False,False,False,"[{'category': 'DA', 'name': 'DAC'}, {'category...","{'breakdown': {'total': 6894716.36, 'ether': 3...",NotApplicable,"[{'name': 'Sequencer Failure', 'value': 'Self ...",NaN,Arbitrum One


In [11]:
l2b_activity[l2b_activity['timestamp']>='2024-10-01'].sample(5)

,timestamp,transactions_per_day,slug
28552,2024-10-10,0,astarzkevm
20405,2024-10-08,12447,xlayer
5118,2024-10-29,204117,zksync-era
2846,2024-10-23,813575,optimism
22781,2024-10-07,44782,sorare


In [12]:
l2b_assets[l2b_assets['timestamp']>='2024-10-01'].sample(5)

,timestamp,native,canonical,external,ethPrice,slug
4415,2024-11-07,1.367088e+08,2.544082e+08,6.613221e+08,2717.5369,scroll
33814,2024-10-08,0.000000e+00,1.189662e+04,1.096077e+06,2428.9380,xchain
35595,2024-10-21,0.000000e+00,6.183960e+05,0.000000e+00,2747.2358,publicgoodsnetwork
34678,2024-10-27,0.000000e+00,6.715789e+05,0.000000e+00,2476.5178,dbk
40636,2024-10-06,0.000000e+00,1.377678e+04,0.000000e+00,2415.4038,deri


### Run Defillama
[Defillama](https://defillama.com/) specializes in TVL (total value locked) data, and also provides high-level metrics for a nearly exhaustive list of chains and applications across ecosystems.

See [API Docs](https://defillama.com/docs/api)

In [13]:
print('getting defillama chain list')
dfl_chains = dfl.get_chain_list()
print('getting defillama chain tvls')
dfl_tvl = dfl.get_all_chains_historical_tvl(dfl_chains)
#filter dates
dfl_tvl = dfl_tvl[dfl_tvl['date']>=start_date]
# filter tvl 
dfl_tvl = dfl_tvl[dfl_tvl['tvl']>0]
print('yay')


getting defillama chain list
getting defillama chain tvls
Defillama API, Chains to run: 314
25 / 314 completed
50 / 314 completed
75 / 314 completed
100 / 314 completed
125 / 314 completed
150 / 314 completed
175 / 314 completed
200 / 314 completed
225 / 314 completed
250 / 314 completed
275 / 314 completed
300 / 314 completed
yay


In [14]:
num_chains = len(dfl_chains)
print(f"Defillama: Total Amount of Chains: {num_chains}")
dfl_chains.sample(5)

Defillama: Total Amount of Chains: 314


,name,chainId
68,Terra Classic,None
71,NEO,None
260,MEER,None
76,Sifchain,None
91,Injective,None


In [15]:
dfl_tvl[dfl_tvl['date']>'2024-10-01'].sample(5)

,date,tvl,name,chain_id
160190,2024-10-19,1.419996e+04,MultiVAC,None
19983,2024-10-09,9.253332e+08,Polygon,137
204191,2024-10-03,9.439074e+05,RENEC,None
67153,2024-11-05,2.144370e+07,Bifrost Network,None
191387,2024-10-13,4.690061e+06,Dexalot,432204


### Export Data
For this demo, we'll export to local csv files.

In practice, you could also push files to a database

In [16]:
folder_name = 'downloads'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create it
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'downloads' already exists.


In [17]:
# Temporary function to store our exporting format more cleanly
def write_df_to_csv(df, file_name):
    df.to_csv(f'{folder_name}/{file_name}.csv', index=False)

#Export Files
print('export growthepie')
write_df_to_csv(gtp_md,'growthepie_metadata')
write_df_to_csv(gtp_fun,'growthepie_fundamentals')

print('export l2beat')
write_df_to_csv(l2b_summary,'l2beat_summary')
write_df_to_csv(l2b_activity,'l2beat_activity')
write_df_to_csv(l2b_assets,'l2beat_assets_onchain')

print('export defillama')
write_df_to_csv(dfl_chains,'defillama_chains')
write_df_to_csv(dfl_tvl,'defillama_tvl')

print('yay')

export growthepie
export l2beat
export defillama
yay
